# Notebook to compute metrics and create charts/tables for useful financial metrics for public equities

- Time Series charting
- Comparing metrics across growth equities
- Calculating Shiller P/E
- Calculating sharpe ratio
- Calculating free related earning metrics


In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# P/E and forward P/E and other metrics


`Revenue Growth`

Definition: Revenue growth measures the percentage increase (or decrease) in a company’s sales over a specified period, typically compared year-over-year. It is expressed as a decimal in yfinance (e.g., 0.15 represents 15% growth).

Interpretation:

Positive Revenue Growth: Indicates that the company’s sales are increasing, which is generally a positive sign, showing that the company is expanding its market presence, increasing demand, or launching successful products.
High Growth (> 10%): Often seen in rapidly expanding industries or companies introducing new products or entering new markets (e.g., technology, biotech).
Moderate Growth (2%–10%): Stable growth, typical in mature or stable industries (e.g., utilities, consumer goods).
Negative Revenue Growth: Indicates that the company’s sales are declining. This could be due to industry downturns, loss of market share, economic conditions, or other challenges.
Moderate Decline (< -5%): May be manageable or temporary, but it's important to investigate the reasons.
Significant Decline (> -10%): Can indicate underlying issues with demand or competitive pressure, and could be a red flag for investors.
Use in Analysis: Revenue growth is essential for evaluating growth-stage companies and understanding whether a company is expanding its business effectively. Investors often prefer companies with steady revenue growth as it can lead to increased profitability over time.

`Beta`

Definition: Beta measures the volatility of a stock relative to the overall market. The market itself has a beta of 1.0, so a stock’s beta tells you how it behaves compared to the market.

Interpretation:

Beta = 1: The stock’s price tends to move in line with the market. If the market goes up or down by 1%, the stock is expected to do the same.
Beta > 1: Indicates that the stock is more volatile than the market. For example, a beta of 1.5 suggests that the stock might move 1.5% for every 1% change in the market, up or down. Higher beta stocks tend to experience larger price swings, which can be appealing in bull markets but risky in downturns.
Beta < 1: Indicates that the stock is less volatile than the market. For example, a beta of 0.8 suggests the stock is expected to move only 0.8% for every 1% change in the market. Lower-beta stocks tend to be more stable and may appeal to conservative investors or those looking for income stability.
Negative Beta: Rare, but it means the stock moves inversely to the market. Gold or utility stocks can sometimes have a low or negative beta, as they’re considered safer or counter-cyclical assets.
Use in Analysis: Beta is useful for understanding a stock’s risk profile. High-beta stocks can provide greater returns during market upswings but pose more risk during downswings, making them more suitable for risk-tolerant investors. Low-beta stocks are often seen as safer investments and may be attractive to risk-averse investors, especially in volatile markets.

Example Interpretation:
If a tech company has a revenue growth of 0.2 (20%) and a beta of 1.3:

The 20% revenue growth indicates strong demand and expansion potential.
The beta of 1.3 suggests that the stock is more volatile than the market, meaning it’s likely to perform well in strong markets but may pose higher risk during downturns.
Both metrics together can provide insights into the growth potential and risk level of the investment.

The beta values presented by the yfinance API are typically calculated using historical price data and measure a stock's volatility relative to the overall market, usually the S&P 500 as the benchmark index. Here’s a breakdown of how beta is calculated:

Historical Returns Calculation:

Both the stock’s returns and the benchmark index’s returns (e.g., daily or weekly) are calculated over a specific period, usually the past 2–5 years.
Covariance and Variance:

Covariance: The covariance between the stock’s returns and the benchmark index’s returns is calculated. This measure tells us how the returns of the stock and the index move together.
Variance: The variance of the benchmark index’s returns is calculated. This represents the index’s volatility.
Beta Calculation:

The beta value is then computed using the following formula:

Beta = Covariance of the stock and market returns / Variance of the market returns​
 
This results in a ratio that indicates the stock’s sensitivity to market movements.

Interpretation of Beta from yfinance:
Beta > 1: Stock is more volatile than the market. A beta of 1.5, for example, suggests the stock tends to move 1.5% for every 1% movement in the market.
Beta < 1: Stock is less volatile than the market. A beta of 0.8 suggests it will move 0.8% for every 1% market movement.
Beta = 1: Stock’s price moves in line with the market.
Practical Note:
The exact lookback period (e.g., 2 or 5 years) and frequency of returns (daily, weekly, monthly) used in yfinance for calculating beta can vary and may not always be explicitly detailed, as yfinance retrieves data directly from Yahoo Finance, which does not disclose their specific beta calculation methodology.

In [ ]:
import yfinance as yf
import pandas as pd

with open("tickers.txt", "r") as file:
    tickers = [line.strip() for line in file]

# Initialize an empty list to store data
data = []

# Collect the P/E and forward P/E for each company
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        company_name = stock.info.get("longName", "N/A")
        pe_ratio = stock.info.get("trailingPE", "N/A")
        forward_pe = stock.info.get("forwardPE", "N/A")
        revenue_growth = stock.info.get("revenueGrowth", "N/A")
        beta = stock.info.get("beta", "N/A")
        data.append({"Ticker": ticker, 
                     "Company Name": company_name, 
                     "P/E Ratio": pe_ratio, 
                     "Forward P/E": forward_pe, 
                     "Revenue Growth": revenue_growth, 
                     "Beta": beta})
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Load data into a DataFrame
df = pd.DataFrame(data)

display(df)

# Load data on fund performance

In [ ]:
import pandas as pd
import plotly.graph_objs as go

# Load your CSV file
df = pd.read_csv('./NAV_Base_20NOV2024.csv')

# Convert 'ReportDate' to datetime format
df['ReportDate'] = pd.to_datetime(df['ReportDate'], format='%Y%m%d')

# Calculate the 10-day moving average
df['10_Day_MA'] = df['Total'].rolling(window=10).mean()

# Create the Plotly figure
fig = go.Figure()

# Add the daily time series line
fig.add_trace(go.Scatter(x=df['ReportDate'], y=df['Total'], mode='lines', name='Daily Total'))

# Add the 10-day moving average line
fig.add_trace(go.Scatter(x=df['ReportDate'], y=df['10_Day_MA'], mode='lines', name='10-Day Moving Average'))

# Update layout
fig.update_layout(
    title="Total Amount Over Time with 10-Day Moving Average",
    xaxis_title="Date",
    yaxis_title="Total (in dollars)",
    template="plotly_white"
)

# Display the plot
fig.show()

# Create comparables and plot charts


In [ ]:
# Convert 'ReportDate' to datetime format
df['ReportDate'] = pd.to_datetime(df['ReportDate'], format='%Y%m%d')

# Filter data from December 15, 2023, onwards
df = df[df['ReportDate'] >= "2023-12-15"].reset_index(drop=True)

# Calculate the 10-day moving average for the 'Total' column
df['10_Day_MA_Total'] = df['Total'].rolling(window=10, min_periods=1).mean()

# Download stock data for SPY and BRK-B starting from Dec 15, 2023
tickers = ["SPY", "BRK-B", "RSP", "QQQE"]
stock_data = yf.download(tickers, start="2023-12-15", end=df['ReportDate'].max())['Adj Close']

# Reset index and rename columns for clarity
stock_data = stock_data.reset_index()
stock_data.columns = ["ReportDate", "SPY", "BRK-B", "RSP", "QQQE"]

# Remove timezone from ReportDate in stock_data to match df
stock_data['ReportDate'] = stock_data['ReportDate'].dt.tz_localize(None)

# Merge stock data with your existing data on 'ReportDate'
df = pd.merge(df, stock_data, on="ReportDate", how="left")

# Calculate the 10-day moving averages for SPY and BRK-B
df['10_Day_MA_SPY'] = df['SPY'].rolling(window=10, min_periods=1).mean()
df['10_Day_MA_QQQE'] = df['QQQE'].rolling(window=10, min_periods=1).mean()
df['10_Day_MA_RSP'] = df['RSP'].rolling(window=10, min_periods=1).mean()
df['10_Day_MA_BRK-B'] = df['BRK-B'].rolling(window=10, min_periods=1).mean()

# Use the first values on Dec 15, 2023, for normalization
initial_total = df['Total'].iloc[0]
initial_ma_total = df['10_Day_MA_Total'].iloc[0]
initial_spy = df['SPY'].iloc[0]
initial_ma_spy = df['10_Day_MA_SPY'].iloc[0]
initial_brkb = df['BRK-B'].iloc[0]
initial_ma_brkb = df['10_Day_MA_BRK-B'].iloc[0]
initial_rsp = df['RSP'].iloc[0]
initial_ma_rsp = df['10_Day_MA_RSP'].iloc[0]
initial_qqqe = df['QQQE'].iloc[0]
initial_ma_qqqe = df['10_Day_MA_QQQE'].iloc[0]

# Normalize all series to start at 100 on Dec 15, 2023
df['Total_Norm'] = (df['Total'] / initial_total) * 100
df['10_Day_MA_Total_Norm'] = (df['10_Day_MA_Total'] / initial_ma_total) * 100
df['SPY_Norm'] = (df['SPY'] / initial_spy) * 100
df['10_Day_MA_SPY_Norm'] = (df['10_Day_MA_SPY'] / initial_ma_spy) * 100
df['BRK-B_Norm'] = (df['BRK-B'] / initial_brkb) * 100
df['10_Day_MA_BRK-B_Norm'] = (df['10_Day_MA_BRK-B'] / initial_ma_brkb) * 100
df['RSP_Norm'] = (df['RSP'] / initial_rsp) * 100
df['10_Day_MA_RSP_Norm'] = (df['10_Day_MA_RSP'] / initial_ma_rsp) * 100
df['QQQE_Norm'] = (df['QQQE'] / initial_qqqe) * 100
df['10_Day_MA_QQQE_Norm'] = (df['10_Day_MA_QQQE'] / initial_ma_qqqe) * 100

# Create the Plotly figure
fig = go.Figure()

# Add normalized daily and 10-day moving average lines for 'Total'
fig.add_trace(go.Scatter(x=df['ReportDate'], y=df['Total_Norm'], mode='lines', name='Daily Total (Normalized)'))
fig.add_trace(go.Scatter(x=df['ReportDate'], y=df['10_Day_MA_Total_Norm'], mode='lines', name='10-Day MA Total (Normalized)'))

# Add normalized daily and 10-day moving average lines for SPY
#fig.add_trace(go.Scatter(x=df['ReportDate'], y=df['SPY_Norm'], mode='lines', name='SPY (Normalized)'))
fig.add_trace(go.Scatter(x=df['ReportDate'], y=df['10_Day_MA_SPY_Norm'], mode='lines', name='10-Day MA SPY (Normalized)'))
fig.add_trace(go.Scatter(x=df['ReportDate'], y=df['10_Day_MA_BRK-B_Norm'], mode='lines', name='10-Day MA BRK-B (Normalized)'))
fig.add_trace(go.Scatter(x=df['ReportDate'], y=df['10_Day_MA_RSP_Norm'], mode='lines', name='10-Day MA RSP (Normalized)'))
fig.add_trace(go.Scatter(x=df['ReportDate'], y=df['10_Day_MA_QQQE_Norm'], mode='lines', name='10-Day MA QQQE (Normalized)'))

# Update layout
fig.update_layout(
    title="Normalized Total, 10-Day Moving Averages, SPY, and BRK-B Over Time (Starting Dec 15, 2023)",
    xaxis_title="Date",
    yaxis_title="Normalized Value (Starting at 100)",
    template="plotly_white"
)

# Display the plot
fig.show()

# Constructing a demo portfolio 

## Collect the data

In [ ]:
import yfinance as yf

# Define the stock tickers
tickers = ["RSP", "QQQE", "SPY", "BRK-B"] 

MA_WINDOW = 30 #number of days for the moving average window
START_DATE = "2023-11-09"
END_DATE = "2024-11-08"

# Download historical data for the last 5 years, including dividends
data = yf.download(tickers, start=START_DATE, end=END_DATE, actions=True)['Adj Close']

# Calculate the N-day moving average
data_ma = data.rolling(window=MA_WINDOW).mean()

# Normalize the data to start at 100 for easy comparison
normalized_data_ma = data_ma / data_ma.iloc[0] * 100  # start normalization after the 7th day

# Create the synthetic fund with the "PEF" ticker as an equally weighted average of the other tickers (excluding SPY)
# Normalize the data to start at 100 for easy comparison
normalized_data = data / data.iloc[0] * 100

# Create the "PEF" ticker as an equally weighted portfolio at the start
initial_weights = 1 / len(tickers[:-2])  # Exclude SPY and BRKB from the calculation
pef_series = (normalized_data[tickers[:-2]] * initial_weights).sum(axis=1)

# Calculate the N-day moving average for all stocks including "PEF"
normalized_data_ma["SYN-FUND1"] = normalized_data_ma[tickers[:-2]].mean(axis=1) #
normalized_data_ma["SYN-FUND2"] = pef_series.rolling(window=MA_WINDOW).mean() #

# Add BRKB to the tickers for plotting
tickers_with_pef = tickers + ["SYN-FUND1", "SYN-FUND2"]

In [ ]:
data_ma

## Plot the data

In [ ]:
import plotly.graph_objects as go

# Create the plot
fig = go.Figure()

# Add traces for each stock
for ticker in tickers:
    fig.add_trace(go.Scatter(x=normalized_data_ma.index, 
                             y=normalized_data_ma[ticker],
                             mode='lines', 
                             name=ticker))

# Add the PEF trace with a dotted black line
fig.add_trace(go.Scatter(x=normalized_data_ma.index, 
                         y=normalized_data_ma["SYN-FUND1"],
                         mode='lines', 
                         name="SYN-FUND1", 
                         line=dict(color='black', width=4, dash='dot')))

fig.add_trace(go.Scatter(x=normalized_data_ma.index, 
                         y=normalized_data_ma["SYN-FUND2"],
                         mode='lines', 
                         name="SYN-FUND2", 
                         line=dict(color='grey', width=4, dash='dot')))

# Customize the layout
fig.update_layout(
    title="Performance of Selected Stocks Including Dividends ("+str(MA_WINDOW)+"-Day Moving Average)",
    xaxis_title="Date",
    yaxis_title="Normalized Price (Starting at 100)",
    legend_title="Stocks",
    hovermode="x unified",
    width=1400,  # Set width to 14 inches (14 * 100 pixels per inch)
    height=800,  # Set height to 8 inches (8 * 100 pixels per inch)
    plot_bgcolor='white',  # Set the plot background to white
    paper_bgcolor='white'  # Set the outer background to white
)

# Show the interactive plot
fig.show()

# Shiller P/E

The Shiller P/E Ratio, also known as the Cyclically Adjusted Price-to-Earnings (CAPE) Ratio, is calculated by dividing the current stock price by the average inflation-adjusted earnings of the company over the last 10 years. While the yfinance API does not directly provide the Shiller P/E Ratio, you can approximate it by combining historical data for price and earnings.

Steps to Calculate Shiller P/E Using yfinance
Retrieve Historical Prices: Get the stock's adjusted close prices over the past 10 years.
Retrieve Historical Earnings Per Share (EPS):
Use yfinance to fetch quarterly or annual financial data to calculate historical EPS.
Adjust EPS for inflation (e.g., using CPI data from external sources).
Calculate 10-Year Average Real EPS: Compute the average of inflation-adjusted EPS over the last 10 years.
Calculate Shiller P/E: Divide the current price by the 10-year average EPS.

The Shiller P/E Ratio is most valuable for long-term investors who want to assess a stock's valuation in the context of historical performance and economic cycles. It complements the Current P/E and Forward P/E, which are better suited for short-term decisions and growth-focused evaluations. Together, these metrics provide a comprehensive picture of a stock's valuation and growth potential. This metric helps investors assess whether a stock is overvalued or undervalued based on normalized, inflation-adjusted earnings. This is particularly useful for long-term investors aiming to predict returns over extended periods.

Summary Table of Differences
Metric	| Focus		| Strengths	| 	Weaknesses
Shiller P/E		| 10-Year Avg. Earnings		| Long-term perspective, smoothed trends, inflation-adjusted	| 	Less responsive to recent changes
Current P/E		| Most Recent Earnings		| Simple, widely available, reflects current conditions		| Sensitive to short-term anomalies
Forward P/E	| 	Projected Next 12 Months		| Accounts for expected future growth		| Dependent on forecast accuracy

This does not seem to work with yfinance API

In [ ]:
def calculate_shiller_pe(ticker):
    # Step 1: Fetch historical stock data
    stock = yf.Ticker(ticker)
    historical_prices = stock.history(period="1y")['Close']
    current_price = historical_prices.iloc[-1]
    
    # Step 2: Fetch historical EPS data
    # Note: yfinance provides financials for recent years; older EPS data might need manual sourcing.
    try:
        earnings_data = stock.financials.loc["Net Income"] / stock.financials.loc["Total Shares Outstanding"]
        eps_annual = earnings_data.resample('Y').mean()
    except KeyError:
        print("EPS data not available for this stock.")
        return None
    
    # Step 3: Calculate inflation-adjusted 10-year average EPS
    # (For simplicity, assuming no inflation adjustment here; replace with CPI adjustment as needed)
    ten_year_avg_eps = eps_annual.tail(10).mean()
    
    # Step 4: Calculate Shiller P/E Ratio
    if ten_year_avg_eps > 0:
        shiller_pe = current_price / ten_year_avg_eps
    else:
        shiller_pe = None
    
    return shiller_pe

# Example usage
ticker = "AAPL"  # Replace with your stock ticker
shiller_pe = calculate_shiller_pe(ticker)
print(f"Shiller P/E Ratio for {ticker}: {shiller_pe}")

# Sharpe Ratio

The Sharpe Ratio is a widely used measure of risk-adjusted return, helping investors understand how much return they are earning per unit of risk taken on a stock, portfolio, or investment strategy. It compares the excess return (return above the risk-free rate) to the total risk (measured by standard deviation) of the investment.

Sharpe Ratio  =  (Rp - Rf) / sigma

Rp = average return of stock
Rf =  risk free rate of return (yield on gov bond)
sigma = standard deviation of stock's return

What the Sharpe Ratio Tells You
Risk-Adjusted Return:

The Sharpe Ratio quantifies how much excess return you are earning per unit of risk.
A higher Sharpe Ratio indicates a more favorable risk-adjusted return.
Comparison Across Investments:

It allows investors to compare the performance of different stocks, funds, or portfolios on a level playing field by accounting for risk.
Example: A stock with a 10% return and a high volatility (standard deviation) may have a lower Sharpe Ratio than one with an 8% return but lower volatility.
Portfolio Optimization:

Used in portfolio construction to evaluate whether adding a stock or asset improves the portfolio's overall risk-adjusted performance.
Interpreting the Sharpe Ratio
Sharpe Ratio > 1: Indicates a good risk-adjusted return. The investment is providing a higher return for the level of risk taken.
Sharpe Ratio = 1: Indicates that the investment’s return is proportional to the risk taken.
Sharpe Ratio < 1: Indicates a suboptimal risk-adjusted return, where the level of risk may not justify the returns earned.
Negative Sharpe Ratio: The stock or portfolio is underperforming the risk-free rate, meaning the investment is not providing any additional value for the risk taken.

Limitations
Assumes Normal Distribution: The Sharpe Ratio assumes returns are normally distributed, which is not always true (e.g., stocks with skewed or fat-tailed distributions).
Sensitive to Time Horizon: Calculating the Sharpe Ratio over different time periods can yield different results.
Focuses on Volatility: It only considers total risk (standard deviation) and doesn’t differentiate between upside and downside volatility.


In [ ]:
def compute_sharpe_ratio(ticker, risk_free_rate=0.03, period="5y", interval="1d"):
    """
    Computes the Sharpe Ratio for a given stock.

    Parameters:
    - ticker: str, Stock ticker symbol (e.g., "AAPL")
    - risk_free_rate: float, Annualized risk-free rate (default is 3%)
    - period: str, Period for fetching historical data (default is 1 year)
    - interval: str, Interval for price data (default is daily)

    Returns:
    - Sharpe Ratio: float
    """
    # Fetch historical data for the stock
    stock = yf.Ticker(ticker)
    historical_data = stock.history(period=period, interval=interval)

    # Calculate daily returns
    historical_data['Daily Return'] = historical_data['Close'].pct_change()

    # Compute average daily return and standard deviation
    avg_daily_return = historical_data['Daily Return'].mean()
    std_dev_daily_return = historical_data['Daily Return'].std()

    # Convert risk-free rate to daily (assuming 252 trading days in a year)
    daily_risk_free_rate = risk_free_rate / 252

    # Compute the Sharpe Ratio
    print(f"Average daily return {ticker}: {avg_daily_return:.4f}")
    print(f"Daily risk free rate {ticker}: {daily_risk_free_rate:.4f}")
    print(f"Std dev of daily return {ticker}: {std_dev_daily_return:.4f}")
    
    sharpe_ratio = (avg_daily_return - daily_risk_free_rate) / std_dev_daily_return

    return sharpe_ratio

# Example usage
ticker = "SHV"  # Replace with the stock ticker
sharpe_ratio = compute_sharpe_ratio(ticker)
print(f"Sharpe Ratio for {ticker}: {sharpe_ratio:.4f}")

# FRE and forward FRE for PE companies

The yfinance API does not provide direct metrics for Fee-Related Earnings (FRE) or Future Fee-Related Earnings (often forward-looking projections) for private equity firms like APO (Apollo Global Management) and KKR (KKR & Co.). However, you can approximate Fee-Related Earnings (FRE) by using a combination of available metrics and financial statements, such as revenue, management fees, and earnings.

Here’s an approach to calculating and estimating FRE based on data available through yfinance and other financial information sources.

Steps to Calculate Fee-Related Earnings (FRE)
Extract Revenue and Fee Information: Use the yfinance API to retrieve revenue and other financial metrics. While exact FRE is not directly available, management fees or advisory fees often serve as a base for estimating fee-related earnings.

Calculate Fee-Related Earnings (Approximation):

FRE can generally be approximated as Operating Income from management fees and other fee-related revenue (e.g., advisory, asset management).
Use yfinance to extract Total Revenue, Operating Income, or Net Income, and estimate based on fee-related business segments.
Estimate Future Fee-Related Earnings:

Estimate future FRE based on historical FRE growth trends, which may be calculated as a percentage increase over the previous period.
For a forward estimate, calculate a conservative growth rate based on company earnings reports or forward guidance.

In [ ]:
with open("tickers_PE.txt", "r") as file:
    tickers = [line.strip() for line in file]

# Initialize a list to store results
data = []

# Define a placeholder for the projected FRE growth rate
projected_fre_growth_rate = 0.1  # For example, assume 10% growth

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        
        # Retrieve general info and financial metrics
        company_name = stock.info.get("longName", "N/A")
        # Retrieve the market cap
        market_cap = stock.info.get("marketCap", "N/A")
        pe_ratio = stock.info.get("trailingPE", "N/A")
        forward_pe = stock.info.get("forwardPE", "N/A")
        revenue_growth = stock.info.get("revenueGrowth", "N/A")
        beta = stock.info.get("beta", "N/A") 

        # Approximating FRE (not exact as per company's reporting standards)
        operating_income = stock.financials.loc["Operating Income"][0] if "Operating Income" in stock.financials.index else "N/A" 
        fre = operating_income if operating_income != "N/A" else "N/A"
        # Calculate P/FRE if both Market Cap and FRE are available
        pfre = market_cap / fre if market_cap != "N/A" and fre != "N/A" else "N/A"
        # Estimate Future FRE (Forward FRE) with a growth assumption
        future_fre = fre * (1 + projected_fre_growth_rate) if fre != "N/A" else "N/A"
        # Calculate P/Forward FRE
        p_forward_fre = market_cap / future_fre if market_cap != "N/A" and future_fre != "N/A" else "N/A"
        
        # Append data to list
        data.append({
            "Ticker": ticker,
            "Company Name": company_name,
            "Revenue Growth": revenue_growth, 
            "Beta": beta,
            "P/E Ratio": pe_ratio, 
            "Forward P/E": forward_pe, 
            "P/FRE": pfre,
            "Forward P/FRE": p_forward_fre
        })
        
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Create a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
display(df)